In [4]:
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
test2 = pd.read_csv('model2_encoded.csv')
test2

,Area,Activity,Sex,Age,Injury,Fatal (Y/N),Time,Species
0,158,830,0,50,2407,N,219,1283
1,277,1109,0,2,2215,N,284,1273
2,315,1153,1,40,1407,N,35,1273
3,497,1153,1,80,2217,N,284,235
4,195,579,1,80,1706,N,284,1227
...,...,...,...,...,...,...,...,...
6297,806,274,1,80,670,Y,284,1273
6298,806,843,1,80,670,Y,284,1273
6299,508,1190,1,80,670,Y,284,1273
6300,763,1409,1,80,670,Y,284,1273


In [7]:
X = test2.drop(labels='Fatal (Y/N)', axis=1)

y = test2['Fatal (Y/N)']
from sklearn.model_selection import train_test_split
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=1)

#from sklearn.linear_model import LogisticRegression
#classifier = LogisticRegression()
#classifier.fit(X_train, y_train)

from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=2, random_state=0)

clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
prediction_df = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
#print(f"Training Data Score: {classifier.score(X_train, y_train)}")
#print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

#predictions = classifier.predict(X_test)
#pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)

Accuracy: 0.7755749405233942
